<a href="https://colab.research.google.com/github/Chocoding1/Machine_Learning_Deep_Learning/blob/main/05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **교차 검증과 그리드 서치**

지금까지는 모델의 성능을 확인하기 위해 테스트 세트를 사용했다. 그러나 성능 확인을 위해 테스트 세트를 자꾸 사용하다 보면 모델이 점점 테스트 세트에 맞춰진다.<br>
테스트 세트로 모델의 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다. 모델을 만들고 나서 마지막에 딱 한 번만 사용하는 것이 좋다.

## 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대 적합인지 과소 적합인지 판단하기 어렵다.<br>
테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것이다.<br>
이 데이터를 **검증 세트**라고 한다.<br>

1절에서 전체 데이터 중 20%를 테스트 세트로 만들고 나머지를 훈련 세트로 만들었는데, 검증 세트는 이 훈련 세트에서 다시 20%를 떼어 내어 만든다.<br>
1. 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가
2. 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 선정
3. 결정된 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련
4. 마지막으로 테스트 세트에서 최종 점수 평가

In [ ]:
# 데이터 세트 불러오기
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
# 입력 데이터와 타깃 데이터 설정하고 훈련을 위해 넘파이 배열로 변경
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
# 훈련 세트와 데이터 세트 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
# 훈련 세트를 다시 분리하여 검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
# 훈련 세트, 검증 세트, 테스트 세트 크기 확인
print(sub_input.shape, val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [ ]:
# 훈련 세트와 검증 세트를 사용해 모델 훈련
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


위 모델은 확실히 과대 적합 -> 매개변수를 바꾸며 더 좋은 모델을 찾아야 한다.

## 교차 검증

검증 세트를 만드느라 훈련 세트가 줄었는데, 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어지는 만큼 검증 세트로 인해 성능이 좋지 못한 모델이 만들어질 수도 있다. 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불안정할 것이다.<br>
이 때 **교차 검증**을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.<br>
교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그 평가한 여러 점수들을 평균하여 최종 검증 점수를 얻는다.

note) **k-폴드 교차 검증**<br>
훈련 세트를 k개로 나누어서 교차 검증을 수행하는 것으로 검증 세트를 계속해서 변경하며 최종적으로 k개의 서로 다른 검증 세트를 가지고 검증할 수 있다.<br>
이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있다.<br>
검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.<br>
보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용한다.

사이킷런에는 cross_validate()라는 교차 검증 함수가 존재한다.<br>
사용법) 평가할 모델 객체를 첫 번째 매개변수로 전달 -> 앞처럼 직접 검증 세트를 떼어내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달<br>
cross_validate() 함수는 기본적으로 5-폴드 교차 검증 수행(cv 매개변수에서 폴드 수 변경 가능)<br>

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01536226, 0.00766182, 0.00808024, 0.00773144, 0.00756741]), 'score_time': array([0.0019753 , 0.00081444, 0.00102592, 0.00109959, 0.00104976]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


반환값) fit_time, score_time, test_score 키를 가진 딕셔너리 반환<br>
처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미<br>
마지막 test_score키에 교차 검증의 점수가 나오며, 교차 검증의 최종 점수는 test_score 키에 담긴 점수들을 평균하여 얻는다.

In [8]:
# 최종 교차 검증 점수 출력
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


주의할 점) cross_validate()는 훈련 세트를 섞어서 폴드를 나누지 않는다.<br>
앞의 train_test_split() 함수는 알아서 전체 데이터를 섞은 후에 나눠줬지만, cross_validate() 함수는 그렇지 않기 때문에 훈련 세트를 섞기 위해 분할기(splitter)를 지정해야 한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해준다.<br>
cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용, 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.

In [9]:
# 위에서 수행한 교차 검증은 다음 코드와 동일
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


위의 결과와 동일

In [11]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면?
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719
